In [1]:
%%capture
# Installs Unsloth, Xformers (Flash Attention) and all other packages!
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

# We have to check which Torch version for Xformers (2.3 -> 0.0.27)
from torch import __version__; from packaging.version import Version as V
xformers = "xformers==0.0.27" if V(__version__) < V("2.4.0") else "xformers"
!pip install --no-deps {xformers} trl peft accelerate bitsandbytes triton

In [2]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 15 trillion tokens model 2x faster!
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # We also uploaded 4bit for 405b!
    "unsloth/Mistral-Nemo-Base-2407-bnb-4bit", # New Mistral 12b 2x faster!
    "unsloth/Mistral-Nemo-Instruct-2407-bnb-4bit",
    "unsloth/mistral-7b-v0.3-bnb-4bit",        # Mistral v3 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3.5-mini-instruct",           # Phi-3.5 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth: Already have LoRA adapters! We shall skip this step.


In [ ]:
import pandas as pd
def read_jsonl(file_path, nrows=None):
    return pd.read_json(file_path, lines=True, nrows=nrows)


train_data = read_jsonl('/content/attribute_train.data')
train_solution = read_jsonl('/content/attribute_train.solution')
test_df = read_jsonl('/content/attribute_test.data')

ValueError: Unmatched ''"' when when decoding 'string'

In [ ]:
n = 10000
m = 50000
val_data = train_data[:n]
val_solution = train_solution[:n]

train_data = train_data[n:n+m]
train_solution = train_solution[n:n+m]

In [ ]:
# Step 1: Create the test_solution DataFrame with the same columns as train_solution
test_solution = pd.DataFrame(columns=train_solution.columns)

# Step 2: Copy the 'indoml_id' column from test_data to test_solution
test_solution['indoml_id'] = test_df['indoml_id']

# Step 3: Fill all other columns with 'test'
for column in test_solution.columns:
    if column != 'indoml_id':
        test_solution[column] = 'test'

# Display the first few rows to verify
test_solution.head()

In [ ]:
def preprocess_data(data, solution):
    merged = pd.merge(data, solution, on='indoml_id')

    merged['input_text'] = merged.apply(lambda row: f"title: {row['title']} store: {row['store']} details_Manufacturer: {row['details_Manufacturer']}", axis=1)
    merged['target_text'] = merged.apply(lambda row: f"details_Brand: {row['details_Brand']} L0_category: {row['L0_category']} L1_category: {row['L1_category']} L2_category: {row['L2_category']} L3_category: {row['L3_category']} L4_category: {row['L4_category']}", axis=1)

    return merged[['input_text', 'target_text']]


train_processed = preprocess_data(train_data, train_solution)
test_processed = preprocess_data(test_df, test_solution)
val_processed = preprocess_data(val_data, val_solution)


In [ ]:
# Convert to Hugging Face Dataset format
from datasets import Dataset, DatasetDict
train_dataset = Dataset.from_pandas(train_processed)
test_dataset = Dataset.from_pandas(test_processed)
val_dataset = Dataset.from_pandas(val_processed)

In [ ]:
# Create the dataset in the correct format
# Prepare instructions, inputs, and outputs
instructions = ["Given the product title, store name, and manufacturer details, predict the brand and the hierarchical categories (L0 to L4) for the product in the specified format."] * len(train_dataset)
inputs = train_dataset['input_text']
outputs = train_dataset['target_text']
train_ds = Dataset.from_dict({
    "instruction": instructions,
    "input": inputs,
    "output": outputs
})

In [ ]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

dataset = train_ds.map(formatting_prompts_func,batched=True)

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

In [ ]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.748 GB.
5.984 GB of memory reserved.


In [ ]:
trainer_stats = trainer.train()

NameError: name 'trainer' is not defined

In [ ]:
import re
def extract_details(text):
    pattern = r'details_Brand: (.?) L0_category: (.?) L1_category: (.?) L2_category: (.?) L3_category: (.?) L4_category: (.)'
    match = re.match(pattern, text)
    if match:
        return tuple(item if item is not None else 'na' for item in match.groups())
    return 'na', 'na', 'na', 'na', 'na', 'na'

In [ ]:
torch.cuda.empty_cache()

In [ ]:
from tqdm import tqdm
FastLanguageModel.for_inference(model)
# batch_size = 128
generated_details = []
target_details = []

eval_data = val_dataset['input_text']
eval_label = val_dataset['target_text']
for i in tqdm(range(0, len(eval_data)), desc="Processing test data"):
    inputs = eval_data[i]
    label = eval_label[i] #you are not going to have this

    inputs = tokenizer(
    [
        alpaca_prompt.format(
            "Given the product title, store name, and manufacturer details, predict the brand and the hierarchical categories (L0 to L4) for the product in the specified format.", # instruction
            inputs, # input
            "", # output - leave this blank for generation!
        )
    ], return_tensors = "pt").to("cuda")

    outputs = model.generate(**inputs, max_new_tokens = 128, use_cache = True)
    outputs = tokenizer.batch_decode(outputs)

    # generated_details.append(extract_details(outputs))
    # target_details.append(extract_details(label))
    gen_text = outputs[0].split('Response:\n')[1].split('|end_of_text|>')[0]

    generated_details.append(extract_details(gen_text))
    target_details.append(extract_details(label))

print('Generated info extracted.............')
len(generated_details)

In [ ]:
from tqdm import tqdm
import torch

batch_size = 16
generated_details = []
target_details = []

eval_data = val_dataset['input_text']
eval_label = val_dataset['target_text']

# Process data in batches
for i in tqdm(range(0, len(eval_data), batch_size), desc="Processing test data"):
    batch_inputs = eval_data[i:i+batch_size]
    batch_labels = eval_label[i:i+batch_size] # This is not used for generation but could be useful for evaluation

    # Prepare the batch input
    batch_tokenized_inputs = tokenizer(
        [
            alpaca_prompt.format(
                "Given the product title, store name, and manufacturer details, predict the brand and the hierarchical categories (L0 to L4) for the product in the specified format.", # instruction
                input_text, # input
                "", # output - leave this blank for generation!
            )
            for input_text in batch_inputs
        ], return_tensors="pt", padding=True, truncation=True).to("cuda")

    # Generate outputs in batches
    with torch.no_grad():
        batch_outputs = model.generate(**batch_tokenized_inputs, max_new_tokens=128, use_cache=True)

    # Decode the batch outputs
    decoded_outputs = tokenizer.batch_decode(batch_outputs, skip_special_tokens=True)

    # Extract details for each sample in the batch
    for j, output in enumerate(decoded_outputs):
        gen_text = output.split('Response:\n')[1].split('|end_of_text|>')[0]

        generated_details.append(extract_details(gen_text))
        target_details.append(extract_details(batch_labels[j]))  # if you intend to keep track of labels

    del batch_tokenized_inputs
    del batch_outputs
    torch.cuda.empty_cache()

print('Generated info extracted.............')
print(len(generated_details))


NameError: name 'val_dataset' is not defined